### Steps for quickstart of Agents
- Load the following items: 
    - WebBaseLoader
    - FIASS
    - Embeddings
    - RecursiveCharacterTextSplitter
    - retriever tool

- Using DuckDuckGoSearchRun we created an agent

In [12]:
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.tools.retriever import create_retriever_tool
from langchain_community.document_loaders import WebBaseLoader 
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.agents import AgentExecutor, create_tool_calling_agent, tool

In [2]:
search = DuckDuckGoSearchRun()
llm = ChatOpenAI(model = "gpt-3.5-turbo-0125", temperature= 0)
embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size =1000, 
                                           chunk_overlap = 200).split_documents(docs)
vector = Chroma.from_documents(documents,embedding= embeddings)
retriever = vector.as_retriever()


In [3]:
retriever_tool = create_retriever_tool(
    retriever, 
    "langsmith_search",
    "Search for information about LangSmith. For any question about langSmith, you must use this tool!",
)

In [4]:
tools = [search, retriever_tool]

In [5]:
search.run("manchester united vs bournemouth match summary")

"Match Summary. Sat 9 Dec 2023. Premier League. Manchester United 0 , AFC Bournemouth 3 at Full time. ... Match ends, Manchester United 0, Bournemouth 3. Full Time at 90 minutes plus 7. 90'+7. Report and free match highlights as Man Utd cling onto 2-2 Premier League draw at Bournemouth; Bruno Fernandes twice scores to cancel out goals from Dominic Solanke and Justin Kluivert; Man Utd 10 ... Sunday 10 December 2023 08:07, UK. FREE TO WATCH: Highlights from the Premier League match between Manchester United and Bournemouth. Manchester United crumbled to a humiliating 3-0 defeat against ... Bournemouth vs Manchester United 2-2: Premier League - as it happened. Follow our build-up and text commentary as Bournemouth entertain Manchester United in the Premier League. Manchester United have just one win in seven Premier League games after they drew 2-2 to Bournemouth on Saturday evening. Dominic Solanke gave the Cherries the lead after just 16 minutes when ..."

In [7]:
prompt = hub.pull("hwchase17/openai-functions-agent")

In [8]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [16]:
agent = create_tool_calling_agent(llm= llm , tools= tools, prompt= prompt)

In [17]:
agent_executor = AgentExecutor(agent=agent, tools= tools, verbose=True)
agent_executor.invoke({"input":"Hi"})



> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


{'input': 'Hi', 'output': 'Hello! How can I assist you today?'}

In [18]:
agent_executor.invoke({"input":"Who is barack obama"})



> Entering new AgentExecutor chain...

Invoking: `duckduckgo_search` with `{'query': 'Barack Obama'}`


Learn about the life and achievements of Barack Obama, the first Black commander-in-chief of the United States. From his early years in Hawaii and Indonesia to his political career in Illinois and Washington, D.C., discover his family background, education, marriage, hobbies, and legacy. As Barack Obama was sworn in for his second term, FRONTLINE took a probing look at the first four years of his presidency. (Aired 2013) This journalism is ma... Former President Barack Obama, in a CNN interview that aired Thursday night, said he does not believe President Joe Biden will face a serious primary challenge during his 2024 reelection campaign ... What makes Mr. Obama's presidency distinctive is the way it resonated around the world in the "Obama moment," as Evan McCormick, who led the foreign policy part of the project, put it. WASHINGTON — During a trip to the White House in June, form

{'input': 'Who is barack obama',
 'output': "Barack Obama is the first Black commander-in-chief of the United States. He served as the 44th President of the United States from 2009 to 2017. Obama was born in Hawaii and had a successful political career in Illinois before becoming a prominent figure in Washington, D.C. He is known for his family background, education, marriage, hobbies, and legacy. Obama's presidency was marked by significant events and policies that shaped the country during his tenure."}

In [19]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})




> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'how can LangSmith help with testing'}`


Getting started with LangSmith | 🦜️🛠️ LangSmith

Evaluations in LangSmith" },    ],    datasetId: dataset.id,});// Define your evaluatorconst exactMatch = async ({ run, example }: { run: Run; example?: Example; }): Promise<EvaluationResult> => {    return {        key: 'exact_match',        score: run.outputs?.output === example?.outputs?.output ? 1 : 0,    };};await runOnDataset(    (input: { postfix: string }) => ({ output: `Welcome ${input.postfix}` }), // Your AI system goes here    datasetName, // The data to predict and grade over    {        evaluationConfig: { customEvaluators: [exactMatch] },        projectMetadata: {            version: "1.0.0",            revision_id: "beta",        },    });See more on the evaluation quick start page.Next Steps​Check out the following sections to learn more about LangSmith:User Guide: Learn about the workflows Lang

{'input': 'how can langsmith help with testing?',
 'output': 'LangSmith can help with testing by providing evaluation capabilities for your AI systems. You can define datasets with test cases, create examples with inputs and expected outputs, and use evaluators to score the results. LangSmith supports workflows at each stage of the LLM application lifecycle, offers pricing information, self-hosting options, proxy capabilities, tracing capabilities, and more. Additionally, you can explore the LangSmith Cookbook for tutorials and walkthroughs, the LangChain Python and JS documentation, and join the LangChain Discord for discussions.'}

## Adding memory

In [22]:
agent_executor.invoke({"input":"Hi! my name is Ajeet", "chat_history":[]})



> Entering new AgentExecutor chain...
Hello Ajeet! How can I assist you today?

> Finished chain.


{'input': 'Hi! my name is Ajeet',
 'chat_history': [],
 'output': 'Hello Ajeet! How can I assist you today?'}

In [23]:
from langchain_core.messages import HumanMessage, AIMessage
agent_executor.invoke({
    "chat_history":[
        HumanMessage(content= "Hi! My name is Ajeet Acharya"),
        AIMessage(content = "hello bob how's it going?")
    ],
    "input":"what's my name?"
})



> Entering new AgentExecutor chain...
Your name is Ajeet Acharya.

> Finished chain.


{'chat_history': [HumanMessage(content='Hi! My name is Ajeet Acharya'),
  AIMessage(content="hello bob how's it going?")],
 'input': "what's my name?",
 'output': 'Your name is Ajeet Acharya.'}

In [25]:
agent_executor.invoke({"input":"What is my name?"})



> Entering new AgentExecutor chain...
I'm sorry, but I don't have access to your personal information such as your name. How can I assist you today?

> Finished chain.


{'input': 'What is my name?',
 'output': "I'm sorry, but I don't have access to your personal information such as your name. How can I assist you today?"}